In [2]:
from divisiveclustering.datautils import DataUtils
from divisiveclustering.coresetsUtils import Coreset, normalize_np ,coreset_to_graph
from divisiveclustering.vqe_utils import kernel_two_local, create_Hamiltonian_for_K2, get_Hamil_variables
from divisiveclustering.quantumutils import get_probs_table
import cudaq
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [11]:
number_of_qubits = 5
circuit_depth = 1
parameter_count = 4 * circuit_depth * number_of_qubits 


In [12]:
data_utils = DataUtils('../../data')

In [13]:
try:
    raw_data = data_utils.load_dataset()
except:
    raw_data = data_utils.create_dataset(n_samples = 1000)
    


Data loaded from ../../data/dataset.pickle


In [14]:
coresets = Coreset()

In [15]:
coreset_vectors, coreset_weights = coresets.get_coresets(
    data_vectors=raw_data, number_of_runs=10, coreset_numbers=5, size_vec_list=10
)

In [16]:
best_coreset_vectors, best_coreset_weights = coresets.get_best_coresets(raw_data, coreset_vectors, coreset_weights)

In [17]:
normalized_cv = normalize_np(best_coreset_vectors, centralize=True)
normalized_cw = normalize_np(best_coreset_weights, centralize=False)

In [18]:
coreset_points, G, H, weight_matrix, weights = coreset_to_graph(normalized_cv, normalized_cw, metric="dot")

In [19]:
H = create_Hamiltonian_for_K2(G, number_of_qubits, weights)

In [27]:
optimizer = cudaq.optimizers.COBYLA()
optimizer.initial_parameters = np.random.uniform(-np.pi / 8.0, np.pi / 8.0,
                                                 parameter_count)

optimal_expectation, optimal_parameters = cudaq.vqe(
    kernel=kernel_two_local(number_of_qubits, circuit_depth),
    spin_operator=H,
    optimizer=optimizer,
    parameter_count=parameter_count,
    shots = 100)

counts = cudaq.sample(kernel_two_local(number_of_qubits, circuit_depth), optimal_parameters, shots_count = 100)

print(counts)

-0.518335
<H> = -0.518335
<H> = -0.518336
<H> = -0.518336
<H> = -0.518335
<H> = -0.518336
<H> = -0.518335
<H> = -0.518336
<H> = -0.518335
<H> = -0.518336
<H> = -0.518335
<H> = -0.518336
<H> = -0.518335
<H> = -0.518336
<H> = -0.518335
<H> = -0.518336
<H> = -0.518336
<H> = -0.518336
<H> = -0.518336
<H> = -0.518336
<H> = -0.518336
<H> = -0.518336
<H> = -0.518336
<H> = -0.518336
<H> = -0.518336
<H> = -0.518336
<H> = -0.518336
<H> = -0.518336
<H> = -0.518336
<H> = -0.518336
<H> = -0.518336
<H> = -0.518336
<H> = -0.518336
<H> = -0.518335
<H> = -0.518336
<H> = -0.518336
<H> = -0.518336
<H> = -0.518336
<H> = -0.518336
<H> = -0.518336
<H> = -0.518336
<H> = -0.518336
<H> = -0.518336
<H> = -0.518335
<H> = -0.518336
<H> = -0.518336
<H> = -0.518336
<H> = -0.518335
<H> = -0.339149
<H> = -0.339149
<H> = -0.234744
<H> = -0.339162
<H> = -0.230941
<H> = -0.339162
<H> = -0.338879
<H> = -0.339163
<H> = -0.408453
<H> = -0.408454
<H> = -0.401318
<H> = -0.408476
<H> = -0.109905
<H> = -0.408472
<H> = -0.22048

In [28]:
probs_table = get_probs_table(counts, sort_values= True)
best_bitstring = probs_table.iloc[0][0]
print(f"best string is {best_bitstring}")

best string is 01000


In [29]:
probs_table

,bitstring,probability
0,01000,0.91
1,10111,0.09
